# Individual assignment: more wine!

In this assignment you will work with two datasets of Portuguese red and white wine variants. The data is related to red and white variants of the Portuguese "Vinho Verde" wine.

* Paper: https://www.semanticscholar.org/paper/Modeling-wine-preferences-by-data-mining-from-Cortez-Cerdeira/bf15a0ccc14ac1deb5cea570c870389c16be019c


The columns in the datasets are as follows:

1. fixed acidity: most acids involved with wine or fixed or nonvolatile (do not evaporate readily)
2. volatile acidity: the amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste
3. citric acid: found in small quantities, citric acid can add 'freshness' and flavor to wines
4. residual sugar: the amount of sugar remaining after fermentation stops, it's rare to find wines with less than 1 gram/liter and wines with greater than 45 grams/liter are considered sweet
5. chlorides: the amount of salt in the wine
6. free sulfur dioxide: the free form of SO2 exists in equilibrium between molecular SO2 (as a dissolved gas) and bisulfite ion; it prevents microbial growth and the oxidation of wine
7. total sulfur dioxide: amount of free and bound forms of S02; in low concentrations, SO2 is mostly undetectable in wine, but at free SO2 concentrations over 50 ppm, SO2 becomes evident in the nose and taste of wine
8. density: the density of wine is close to that of water depending on the percent alcohol and sugar content
9. pH: describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale
10. sulphates: a wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial and antioxidant
11. alcohol: the percent alcohol content of the wine

There are two datasets, one for red wine and one for white wine. The goal is to model wine quality.

## Grading

The assignment is graded up to 8 points.

There can be a maximum of 2 points of extra credit, according to the `improvement` percentage in the last question:
* If your hyperparameter tuning improves the model by 5% or more, you get **1 point of extra credit.**
* If it improves the model by 10% or more, you get **2 points of extra credit.**
* If your hyperparameter tuning does not improve the model, you get **0 points of extra credit.**

**The maximum grade is 10 (8 + 2 extra credit if applicable).**

## Question 1 (0.5 points)

Load each dataset as a dataframe and create a new column called `type` with the wine type (red or white, type included in the name of the dataset).

Contatenate the two dataframes into a single dataframe called `wine` and display a sample of 5 rows.

In [26]:
import pandas as pd
import numpy as np

data_white = pd.read_csv('winequality-white.csv')
data_red = pd.read_csv('winequality-red.csv')

new_column = 'type'
data_white[new_column] = 'white'
data_red[new_column] = 'red'

wine = pd.concat([data_white,data_red],axis=0)

print(wine.shape)
print(wine.columns)
wine.head()



(6497, 13)
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'type'],
      dtype='object')


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


## Question 2  (0.5 points)

I just realized that the column `type`should not be a string, but a number that represents a category. Change the column `type` to a numerical column and display the first 5 rows.

In [27]:
# LabelEncoder

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

wine_le = wine.copy()

# Select only the categorical columns > type
categorical_columns = wine.select_dtypes(include=['object']).columns
print(categorical_columns.array)

# Iterate over each categorical column and apply LabelEncoder
for column in categorical_columns:
    wine_le[f'{column}_le'] = label_encoder.fit_transform(wine[column])

# Drop the original categorical columns to keep only their encoded version
wine_le = wine_le.drop(categorical_columns, axis=1)

# Rename type column
wine_le = wine_le.rename(columns={'type_le': 'type'})

# Display the first few rows of the transformed DataFrame
wine_le.head()

<NumpyExtensionArray>
['type']
Length: 1, dtype: object


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1


## Question 3 (1 point)

We need to convert this problem into a classification problem.

Before that, we need a categorical target, and for that we will use the `quality` column. But first, analyze the `quality` column and decide how to convert it into a categorical target.

1. Print the unique values of the `quality` column.
2. Plot a histogram of the `quality` column for each type of wine.
3. Based on the analysis, decide and justify which value will be the threshold to convert the `quality` column into a binary target.
    * For example, if you decide that the threshold is 6, then the target will be 1 if the quality is greater than 6 and 0 otherwise.
    * Don't overthink this, just decide a threshold and justify it.
4. Create a new column called `target` with the binary target, considering the threshold you decided.


In [28]:
# categorical_target = quality
#Print the unique values of the `quality` column.
print('The unique values of the `quality` column')
print(wine_le['quality'].unique())

The unique values of the `quality` column
[6 5 7 8 4 3 9]


In [29]:
import plotly.express as px

# Plot a histogram of the `quality` column for each type of wine 
fig = px.histogram(wine_le,
                    x='quality', 
                    color='type', 
                    barmode='overlay', 
                    title='Wine Quality Distribution by Type',
                    labels={'type': 'Wine Type', 'quality': 'Quality Score'}
                    
                    )

custom_labels = {
    '0' : 'Red Wine',
    '1' : 'White Wine'
}

fig.for_each_trace(lambda t: t.update(name = custom_labels[t.name]))

fig.show()

In [30]:
# The treshold is going to be 7, if the wine has =>7 then the quality is 1 otherwise it will be 0
# using numpy here instead of lambda --> (condition, result on true, result on false)
wine_le['target'] = np.where(wine_le['quality']>= 7, 1, 0)

wine_le.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,target
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1,0


## Question 4 (1 point total, 0.125 each new feature)

Let's create some new features.

You have to create 8 new columns in the dataframe `wine`, doesn't matter whether they'll be useful or not, just create them.

Remember, you can use the following operations to create new columns:
* Basic arithmetic operations between columns
* Label encoding of categorical columns
* Binning of numerical columns (convert a continuous variable column into a categorical column)
* Apply a function to a column
...

In [31]:
#8 new columns - i've renamed wine to wine_le in earlier stages to keep the transformed and the original dataframe separate, i'll be using wine_le

wine_le['acidity_ratio'] = wine_le['fixed acidity'] / wine_le['volatile acidity']
wine_le['total_acidity'] = wine_le['fixed acidity'] + wine_le['volatile acidity']
wine_le['density_alcohol_ratio'] = wine_le['density'] / wine_le['alcohol']
wine_le['sulfur_dioxide_ratio'] = wine_le['free sulfur dioxide'] / wine_le['total sulfur dioxide']
wine_le['quality_category'] = wine_le['quality'].apply(lambda x: 'Low' if x <= 4 else ('Medium' if x <= 6 else 'High'))
wine_le['alcohol_level'] = pd.cut(wine_le['alcohol'], bins=[0, 10, 12, np.inf], labels=['Low', 'Medium', 'High'])
wine_le['acidity_level'] = pd.cut(wine_le['total_acidity'], bins=[0, 5, 10, np.inf], labels=['Low', 'Medium', 'High'])
wine_le['residual_sugar_level'] = pd.cut(wine_le['residual sugar'], bins=[0, 5, 15, np.inf], labels=['Low', 'Medium', 'High'])


print(wine_le.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  ...  \
0                 45.0                 170.0   1.0010  3.00       0.45  ...   
1                 14.0                 132.0   0.9940  3.30       0.49  ...   
2                 30.0                  97.0   0.9951  3.26       0.44  ...   
3                 47.0                 186.0   0.9956  3.19       0.40  ...   
4                 47.0                 186.0   0.9956  3.19       0.40  ...   

   type  target  acidity_ratio  total_acidity  density_a

In [32]:
wine_le.shape
print(wine_le.columns)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'type', 'target',
       'acidity_ratio', 'total_acidity', 'density_alcohol_ratio',
       'sulfur_dioxide_ratio', 'quality_category', 'alcohol_level',
       'acidity_level', 'residual_sugar_level'],
      dtype='object')


In [33]:
# On later exercises when training the model I'll need to have numerical values, since I've created some categorical variables in question 4 so I need to convert them 

# Select only the categorical columns
categorical_columns = wine_le.select_dtypes(include=['object','category']).columns
print(categorical_columns.array)



<NumpyExtensionArray>
['quality_category', 'alcohol_level', 'acidity_level', 'residual_sugar_level']
Length: 4, dtype: object


In [34]:
label_encoder = LabelEncoder()
wine_df_model = wine_le.copy()

# Iterate over each categorical column and apply LabelEncoder
for column in categorical_columns:
    wine_df_model[f'{column}_le'] = label_encoder.fit_transform(wine_le[column])

# Drop the original categorical columns to keep only their encoded version
wine_df_model = wine_df_model.drop(categorical_columns, axis=1)

# Display the first few rows of the transformed DataFrame
wine_df_model.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,...,type,target,acidity_ratio,total_acidity,density_alcohol_ratio,sulfur_dioxide_ratio,quality_category_le,alcohol_level_le,acidity_level_le,residual_sugar_level_le
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,...,1,0,25.925926,7.27,0.113750,0.264706,2,1,2,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,...,1,0,21.000000,6.60,0.104632,0.106061,2,1,2,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,...,1,0,28.928571,8.38,0.098525,0.309278,2,2,2,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,...,1,0,31.304348,7.43,0.100566,0.252688,2,1,2,2
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,...,1,0,31.304348,7.43,0.100566,0.252688,2,1,2,2


## Question 5 (1 point)

Now that we have a target and some new features, we can create a classification model.

But first, we need to remove the `quality` column and split the data into features and target.

1. Remove the `quality` column from the dataframe.
2. Split the data into features and target. Name the features dataframe `x` and the target series `y`.
3. Split the data into training and test sets.
    * Keep in mind the nature of your target, and use stratification if necessary.

In [35]:
print(wine_le.columns)

#Split the data into features and target. Name the features dataframe `x` and the target series `y`.
#Split the data into training and test sets. Keep in mind the nature of your target, and use stratification if necessary.
x = wine_df_model.drop('quality', axis=1)
y = wine_df_model['quality']
print(x.columns)
print(y)


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'type', 'target',
       'acidity_ratio', 'total_acidity', 'density_alcohol_ratio',
       'sulfur_dioxide_ratio', 'quality_category', 'alcohol_level',
       'acidity_level', 'residual_sugar_level'],
      dtype='object')
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'type', 'target', 'acidity_ratio',
       'total_acidity', 'density_alcohol_ratio', 'sulfur_dioxide_ratio',
       'quality_category_le', 'alcohol_level_le', 'acidity_level_le',
       'residual_sugar_level_le'],
      dtype='object')
0       6
1       6
2       6
3       6
4       6
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 6497, dtype: i

In [36]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)


## Question 6  (0.5 points)

Time for scaling the data.

Choose a scaler and scale the features, properly done.

In [37]:
#NA

## Question 7  (0.5 points)

Choose a classification algorithm and initialize it.

In [38]:
from sklearn.ensemble import RandomForestClassifier

tree = RandomForestClassifier()

## Question 8 (1 point)

Test the model's performance on the test set without any hyperparameter tuning, just to see how it performs.

From now on, the metric used to evaluate the model will be the [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html).

Print the F1 score of the model evaluated on the test set.

In [39]:
from sklearn.metrics import f1_score

tree.fit(x_train, y_train)

# test model
y_pred = tree.predict(x_test)

f1_score_value_baseline = f1_score(y_test, y_pred,average='macro')
print('f1 score value - no hyperparameter tuning')
print(f1_score_value_baseline)

# general f1 guidelines:

# 0.90 - 1.00: Excellent
# 0.80 - 0.90: Good
# 0.70 - 0.80: Fair
# 0.60 - 0.70: Poor
# Below 0.60: Needs improvement

# the f1 score indicates an improvement is needed in this case

f1 score value - no hyperparameter tuning
0.5540359680634878


## Question 9 (1 point)

Now we have a baseline to beat, let's try to improve the model's performance and also make it robust by using GridSearchCV.

Create a grid with hyperparameters to be tested and use GridSearchCV to find the best hyperparameters.

Use as many hyperparameters as you want, but test at least 3 different values for each hyperparameter. 

Use no less than 3 folds in GridSearchCV.

Be aware of the time it takes to run GridSearchCV, don't use too many hyperparameters or too many values for each hyperparameter.

Use the grid search fitting time to review your answers above.


In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

tree = RandomForestClassifier(random_state=42)

# Create a parameter grid
param_grid = {
    'n_estimators': [100, 150, 170],
    #'max_depth': [10, 20, 40],
    'min_samples_split': [2, 3, 10],
    #'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    #'max_samples' : [0.5, 0.75, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=4, scoring='f1_macro', n_jobs=-1, verbose=2)

# Fit GridSearchCV to the training data
grid_search.fit(x_train, y_train)

# Retrieve the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Parameters:", best_params)

# Evaluate the best model on the test data
y_pred = best_model.predict(x_test)
f1_score_value = f1_score(y_test, y_pred, average='macro')
print('F1 score value (with hyperparameter tuning):')
print(f1_score_value)

Fitting 4 folds for each of 54 candidates, totalling 216 fits
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=150; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=150; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=150; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=150; total time=   0.0s
[CV] END bootstrap=True, max_features=auto, min_samples_split=2, n_estimators=170; total time=   0.0s
[CV] END bootstrap=T

[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=170; total time=   2.2s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   2.4s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=170; total time=   3.7s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=170; total time=   2.0s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=150; total time=   2.0s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=170; total time=   2.1s
[CV] END bootstrap=True, max_features=sqrt, min_samples_split=2, n_estimators=150;

/home/codespace/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning:


72 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
72 fits failed with the following error:
Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home

Best Parameters: {'bootstrap': False, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
F1 score value (with hyperparameter tuning):
0.5877016942101946


## Question 10 (1 point)

Given the best hyperparameters, train you model with them, and test its performance on the test set.
* Print the best hyperparameters found by GridSearchCV.
* Print the F1 score of the freshly trained model evaluated on the test set.
* Print the improvement in the F1 score compared to the baseline model by using the following relation:

$$ improvement = \frac{F1_{new} - F1_{baseline}}{F1_{baseline}} $$

* Save the predictions in a CSV file called `predictions.csv`.

In [43]:
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")
print(f"F1 Score Value: {f1_score_value}")
print(f"Improvement: {(f1_score_value-f1_score_value_baseline)/f1_score_value_baseline}")

Best Parameters: {'bootstrap': False, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
F1 Score Value: 0.5877016942101946
Improvement: 0.06076451365491314
